#Web Scraping

##Metacritic:
This is an example of a work in progress.

Scraping metacritic, like many webscraping projects, involves two steps.

Parse a main page full of links to other things (like the stories on the front page of a news site, the categories on a retail website), sometimes iteratively, until you have a list of links to your "units" of interst
sometimes these are called "producers" because they produce urls of items you want to gather information about
Parse each atomic unit of interest to get its detailed information.
I often use ipython notebooks to hack through the trial-and-error process of getting my scrapers to work. I start with simple commands or chunks of code (mini-scripts), and debug as I go.

As I go, I collect these into functions.

When I've reached a good stopping point, I copy these functions into a python file, perhaps creating classes, and then I start a new notebook for the next step, import the stuff I made in the first step, and repeat the process on the second (probably atomic) level.

In this notebook,
you see the products of the hacking and function-izing that I did to get movies from a page on metacritic.

On this pass, I get the links to the main pages for each movie, and "while I'm here", I get the information available from the list. I'll add more information about each movie in future steps.

The functions from this notebook
are collected in the file metacritic_scraper.py. You'll see in the next notebook, Data-Con-metacritic2.ipynb, how I import that file and use it to continue hacking on metacritic movie ratings and information.

The final project I included in this repo
is a scraper that collects information about links on datatau.com. It is a (nearly) completed product (nearly) ready to deploy as a cron task so I can see over time how (and which) links gain (or don't) gain votes and ranking on datatau.

To make this work:
"run" each function cell to get it into your namespace, then try it out with the cells at the bottom of the notebook.

In [1]:
import requests
from bs4 import BeautifulSoup
import bs4.element

def connection(url):
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5)'}
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.text)
    return soup

def scrape_metacritic_list_page(soup,movie_list=[]):
    for release_type,mlist in get_modules(soup).items():
        for movie in mlist:
            try:
                m = parse_movie_li(movie)
            except:
                bad.append(movie)
                continue
            m['release_type']=release_type
            movie_list.append(m)
    return movie_list

def just_tags(thinglist):
    tags = [t for t in thinglist if isinstance(t,bs4.element.Tag)]
    return tags
    
def get_modules(soup):
    modules = soup.find_all(class_='product_condensed')
    module_dict = {}
    for mod in modules: 
        label = mod.find(class_='release_type_label').text
        movie_lis = just_tags(mod.find('ol').contents)
        module_dict[label]=movie_lis
    return module_dict
    

from dateutil import parser

def parse_movie_li(li):
    title_div = li.find(class_='product_title')
    
    movie = {
        'title':title_div.text.strip(),
        'rel_url':title_div.find('a')['href'],
        'metascore_w':get_metascore_w(li.find(class_='metascore_w')),
        'release_date': get_release_date(li.find(class_='release_date').find(class_='data'))
    }
    #print movie,'\n'
    return movie
    
def get_metascore_w(div):
    try:
        score = div.text
    except:
        print 'no text in metascore div'
        return None
    try: 
        score = int(score)
    except: 
        pass
    return score
    
    
def get_release_date(div):
    try: 
        datestr = div.text
    except:
        return None
    try: 
        date = parser.parse(datestr)
    except:
        return datestr
    return date

In [2]:
url = 'http://www.metacritic.com/browse/movies/release-date/theaters/date'
soup = connection(url)
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<title>New Movie Releases - Metacritic</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o?o:n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({QJf3ax:[function(e,n){function t(e){function n(n,t,a){e&&e(n,t,a),a||(a={});for(var u=c(n),f=u.length,s=i(a,o,r),p=0;f>p;p++)u[p].apply(s,t);return s}function a(e,n){f[e]=c(e).concat(n)}function c(e){return f[e]||[]}function u(){return t(n)}var f={};return{on:a,emit:n,create:u,listeners:c,_events:f}}function r(){return{}}var o="nr@context",i=e("gos");n.exports=t

In [11]:
movie_list = scrape_metacritic_list_page(soup)
movie_list

[{'metascore_w': 52,
  'rel_url': '/movie/insidious-chapter-3',
  'release_date': datetime.datetime(2015, 6, 5, 0, 0),
  'release_type': u'Wide releases now in theaters',
  'title': u'Insidious: Chapter 3'},
 {'metascore_w': 75,
  'rel_url': '/movie/spy',
  'release_date': datetime.datetime(2015, 6, 5, 0, 0),
  'release_type': u'Wide releases now in theaters',
  'title': u'Spy'},
 {'metascore_w': 38,
  'rel_url': '/movie/entourage',
  'release_date': datetime.datetime(2015, 6, 3, 0, 0),
  'release_type': u'Wide releases now in theaters',
  'title': u'Entourage'},
 {'metascore_w': 40,
  'rel_url': '/movie/aloha',
  'release_date': datetime.datetime(2015, 5, 29, 0, 0),
  'release_type': u'Wide releases now in theaters',
  'title': u'Aloha'},
 {'metascore_w': 43,
  'rel_url': '/movie/san-andreas',
  'release_date': datetime.datetime(2015, 5, 29, 0, 0),
  'release_type': u'Wide releases now in theaters',
  'title': u'San Andreas'},
 {'metascore_w': 47,
  'rel_url': '/movie/poltergeist',
  

In [12]:
from pprint import pprint

#Playing around with the functions

modules = get_modules(soup)
#print type(modules)
#print modules.keys()

label,wide_release_module = modules.items()[0]

#print label
#print wide_release_module

#for item in wide_release_module: print type(item),item.name

#print wide_release_module[0]

movie = parse_movie_li(wide_release_module[0])
pprint(movie)

{'metascore_w': 52,
 'rel_url': '/movie/insidious-chapter-3',
 'release_date': datetime.datetime(2015, 6, 5, 0, 0),
 'title': u'Insidious: Chapter 3'}
